# От нуля до единицы: изучение агентных паттернов

Агенты ИИ. Агентный ИИ. Агентные архитектуры. Агентные рабочие процессы. Агентные паттерны. Агенты повсюду. Но что это такое на самом деле, и как нам создавать надежные и эффективные агентные системы? Хотя термин «агент» используется в широком смысле, ключевой характеристикой является их способность динамически планировать и выполнять задачи, часто используя внешние инструменты и память для достижения сложных целей.

Цель этого поста — изучить распространенные шаблоны проектирования. Думайте об этих шаблонах как о чертежах или многоразовых шаблонах для создания приложений ИИ. Их понимание дает ментальную модель для решения сложных проблем и проектирования систем, которые являются масштабируемыми, модульными и адаптируемыми.

Мы рассмотрим несколько распространенных шаблонов, различая более структурированные **рабочие процессы** и более динамичные **агентные паттерны**. Рабочие процессы обычно следуют предопределенным путям, в то время как агенты имеют больше автономии в принятии решений о своем курсе действий.

**Почему (агентные) паттерны важны?**

*   Паттерны обеспечивают структурированный способ мышления и проектирования систем.
*   Паттерны позволяют нам создавать и развивать сложность приложений ИИ и адаптироваться к меняющимся требованиям. Модульные конструкции, основанные на шаблонах, легче изменять и расширять.
*   Паттерны помогают управлять сложностью координации нескольких агентов, инструментов и рабочих процессов, предлагая проверенные, многоразовые шаблоны. Они продвигают лучшие практики и общее понимание среди разработчиков.

**Когда (и когда не) использовать агентов?**

Прежде чем углубляться в шаблоны, крайне важно рассмотреть, *когда* агентный подход действительно необходим.

*   Всегда ищите самое простое решение в первую очередь. Если вы знаете точные шаги, необходимые для решения проблемы, фиксированный рабочий процесс или даже простой скрипт могут быть более эффективными и надежными, чем агент.
*   Агентные системы часто обменивают повышенную задержку и вычислительные затраты на потенциально лучшую производительность при выполнении сложных, неоднозначных или динамических задач. Убедитесь, что преимущества перевешивают эти затраты.
*   Используйте **рабочие процессы** для предсказуемости и согласованности при работе с четко определенными задачами, где шаги известны. 
*   Используйте **агентов**, когда необходимы гибкость, адаптируемость и принятие решений на основе моделей.
*   Сохраняйте простоту (все еще): даже при создании агентных систем стремитесь к простейшей эффективной конструкции. Чрезмерно сложные агенты могут стать трудными для отладки и управления.
*   Агентность вносит присущую непредсказуемость и потенциальные ошибки. Агентные системы должны включать надежное ведение журнала ошибок, обработку исключений и механизмы повторных попыток, позволяя системе (или базовому LLM) иметь шанс на самокоррекцию.

Ниже мы рассмотрим 3 распространенных шаблона рабочих процессов и 4 агентных шаблона. Мы проиллюстрируем каждый из них, используя чистые вызовы API, не полагаясь на конкретные фреймворки, такие как LangChain, LangGraph, LlamaIndex или CrewAI, чтобы сосредоточиться на основных концепциях.

## Обзор паттернов

Мы рассмотрим следующие паттерны:
- [От нуля до единицы: изучение агентных паттернов](#zero-to-one-learning-agentic-patterns)
  - [Обзор паттернов](#pattern-overview)
  - [Рабочий процесс: цепочка подсказок](#workflow-prompt-chaining)
  - [Рабочий процесс: маршрутизация или передача](#workflow-routing-or-handoff)
  - [Рабочий процесс: распараллеливание](#workflow-parallelization)
  - [Паттерн рефлексии](#reflection-pattern)
  - [Паттерн использования инструментов](#tool-use-pattern)
  - [Паттерн планирования (оркестратор-работники)](#planning-pattern-orchestrator-workers)
  - [Паттерн с несколькими агентами](#multi-agent-pattern)
  - [Комбинирование и настройка этих паттернов](#combining-and-customizing-these-patterns)
  - [Ресурсы:](#resources)

## Рабочий процесс: цепочка подсказок

<div style="display: flex; justify-content: center;">
<img src="../assets/agentic-patterns/prompt-chaining.png" alt="Prompt Chaining" style="max-width: 600px;">
</div>


Вывод одного вызова LLM последовательно передается на вход следующего вызова LLM. Этот шаблон разбивает задачу на фиксированную последовательность шагов. Каждый шаг обрабатывается вызовом LLM, который обрабатывает вывод предыдущего. Он подходит для задач, которые можно четко разбить на предсказуемые, последовательные подзадачи. 

Случаи использования:
*   Создание структурированного документа: LLM 1 создает план, LLM 2 проверяет план на соответствие критериям, LLM 3 пишет контент на основе проверенного плана.
*   Многоэтапная обработка данных: извлечение информации, ее преобразование, а затем обобщение.
*   Создание информационных бюллетеней на основе отобранных входных данных.

In [2]:
import os
from google import genai

# Настройте клиент (убедитесь, что GEMINI_API_KEY установлен в вашей среде)
client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

# --- Шаг 1: Обобщить текст ---
original_text = "Большие языковые модели — это мощные системы ИИ, обученные на огромных объемах текстовых данных. Они могут генерировать человекоподобный текст, переводить языки, писать различные виды творческого контента и отвечать на ваши вопросы в информативной форме."
prompt1 = f"Обобщите следующий текст в одном предложении: {original_text}"

# Используйте client.models.generate_content
response1 = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=prompt1
)
summary = response1.text.strip()
print(f"Резюме: {summary}")

# --- Шаг 2: Перевести резюме ---
prompt2 = f"Переведите следующее резюме на французский, верните только перевод, без другого текста: {summary}"

# Используйте client.models.generate_content
response2 = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=prompt2
)
translation = response2.text.strip()
print(f"Перевод: {translation}")

Резюме: Большие языковые модели — это системы ИИ, обученные на огромных наборах данных для генерации человекоподобного текста, перевода языков, создания контента и предоставления информативных ответов.
Перевод: Les grands modèles linguistiques sont des systèmes d'IA entraînés sur des ensembles de données massifs pour générer du texte de type humain, traduire des langues, créer du contenu et fournir des réponses informatives.


## Рабочий процесс: маршрутизация

<div style="display: flex; justify-content: center;">
<img src="../assets/agentic-patterns/routing-or-handoff.png" alt="Routing" style="max-width: 300px;">
</div>



Начальный LLM действует как маршрутизатор, классифицируя ввод пользователя и направляя его к наиболее подходящей специализированной задаче или LLM. Этот шаблон реализует разделение ответственности и позволяет оптимизировать отдельные последующие задачи (используя специализированные подсказки, разные модели или определенные инструменты) в изоляции. Он повышает эффективность и потенциально снижает затраты за счет использования меньших моделей для более простых задач. Когда задача маршрутизируется, выбранный агент «берет на себя» ответственность за ее выполнение.

Сферы применения:
*   Системы поддержки клиентов: маршрутизация запросов к агентам, специализирующимся на выставлении счетов, технической поддержке или информации о продукте.
*   Многоуровневое использование LLM: маршрутизация простых запросов к более быстрым и дешевым моделям (например, Llama 3.1 8B) и сложных или необычных вопросов к более способным моделям (например, Gemini 1.5 Pro).
*   Генерация контента: маршрутизация запросов на посты в блогах, обновления в социальных сетях или рекламные тексты к различным специализированным подсказкам/моделям.

In [1]:
import os
import json
from google import genai
from pydantic import BaseModel
import enum

# Настройте клиент (убедитесь, что GEMINI_API_KEY установлен в вашей среде)
client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

# Определите схему маршрутизации
class Category(enum.Enum):
    WEATHER = "weather"
    SCIENCE = "science"
    UNKNOWN = "unknown"

class RoutingDecision(BaseModel):
    category: Category
    reasoning: str

# Шаг 1: Маршрутизация запроса
user_query = "Какая погода в Париже?"
# user_query = "Объясните квантовую физику простыми словами."
# user_query = "Какая столица Франции?"

prompt_router = f"""
Проанализируйте запрос пользователя ниже и определите его категорию.
Категории:
- погода: для вопросов о погодных условиях.
- наука: для вопросов о науке.
- неизвестно: если категория неясна.

Запрос: {user_query}
"""

# Используйте client.models.generate_content с конфигурацией для структурированного вывода
response_router = client.models.generate_content(
    model= 'gemini-2.0-flash-lite',
    contents=prompt_router,
    config={
        'response_mime_type': 'application/json',
        'response_schema': RoutingDecision,
    },
)
print(f"Решение о маршрутизации: Категория={response_router.parsed.category}, Обоснование={response_router.parsed.reasoning}")

# Шаг 2: Передача на основе маршрутизации
final_response = ""
if response_router.parsed.category == Category.WEATHER:
    weather_prompt = f"Предоставьте краткий прогноз погоды для местоположения, упомянутого в: '{user_query}'"
    weather_response = client.models.generate_content(
        model='gemini-2.0-flash',
        contents=weather_prompt
    )
    final_response = weather_response.text
elif response_router.parsed.category == Category.SCIENCE:
    science_response = client.models.generate_content(
        model="gemini-2.5-flash-preview-04-17",
        contents=user_query
    )
    final_response = science_response.text
else:
    unknown_response = client.models.generate_content(
        model="gemini-2.0-flash-lite",
        contents=f"Запрос пользователя: {prompt_router}, но на него не удалось ответить. Вот обоснование: {response_router.parsed.reasoning}. Напишите полезный ответ пользователю, чтобы он попробовал еще раз."
    )
    final_response = unknown_response.text
print(f"\nОкончательный ответ: {final_response}")

Решение о маршрутизации: Категория=Category.WEATHER, Обоснование=Запрос касается погодных условий в определенном месте.

Окончательный ответ: Хорошо, учитывая, что вопрос «Какая погода в Париже?», прогноз погоды будет для **Парижа, Франция.**

Вот краткий прогноз:

**Ожидается, что в Париже будет [вставьте сюда реалистичный прогноз, например, переменная облачность с максимумом 22 градуса Цельсия (72 градуса по Фаренгейту) и минимумом 14 градусов Цельсия (57 градусов по Фаренгейту). Днем есть небольшая вероятность дождей].**



## Рабочий процесс: распараллеливание

<div style="display: flex; justify-content: center;">
 <img src="../assets/agentic-patterns/parallelization.png" alt="Parallelization" style="max-width: 300px;">
</div>


Задача разбивается на независимые подзадачи, которые одновременно обрабатываются несколькими LLM, а их результаты агрегируются. Этот шаблон использует параллелизм для задач. Первоначальный запрос (или его части) отправляется нескольким LLM параллельно с отдельными подсказками/целями. После завершения всех ветвей их индивидуальные результаты собираются и передаются конечному агрегатору LLM, который синтезирует их в окончательный ответ. Это может уменьшить задержку, если подзадачи не зависят друг от друга, или повысить качество за счет таких методов, как голосование по большинству или создание разнообразных вариантов.

Сферы применения:
*   RAG с декомпозицией запросов: разбиение сложного запроса на подзапросы, параллельный запуск поиска для каждого и синтез результатов.
*   Анализ больших документов: разделение документа на разделы, параллельное обобщение каждого раздела, а затем объединение резюме.
*   Создание нескольких точек зрения: задание нескольким LLM одного и того же вопроса с разными подсказками-персонами и агрегирование их ответов.
*   Операции в стиле Map-reduce с данными.

In [32]:
import os
import asyncio
import time
from google import genai

# Настройте клиент (убедитесь, что GEMINI_API_KEY установлен в вашей среде)
client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

async def generate_content(prompt: str) -> str:
        response = await client.aio.models.generate_content(
            model="gemini-2.0-flash",
            contents=prompt
        )
        return response.text.strip()

async def parallel_tasks():
    # Определите параллельные задачи
    topic = "дружелюбный робот, исследующий джунгли"
    prompts = [
        f"Напишите короткую, приключенческую идею истории о {topic}.",
        f"Напишите короткую, смешную идею истории о {topic}.",
        f"Напишите короткую, таинственную идею истории о {topic}."
    ]
    # Запустите задачи одновременно и соберите результаты
    start_time = time.time()
    tasks = [generate_content(prompt) for prompt in prompts]
    results = await asyncio.gather(*tasks)
    end_time = time.time()
    print(f"Затраченное время: {end_time - start_time} секунд")

    print("\n--- Индивидуальные результаты ---")
    for i, result in enumerate(results):
        print(f"Результат {i+1}: {result}\n")

    # Агрегируйте результаты и сгенерируйте окончательную историю
    story_ideas = '\n'.join([f"Идея {i+1}: {result}" for i, result in enumerate(results)])
    aggregation_prompt = f"Объедините следующие три идеи истории в один, связный абзац-резюме:{story_ideas}"
    aggregation_response = await client.aio.models.generate_content(
        model="gemini-2.5-flash-preview-04-17",
        contents=aggregation_prompt
    )
    return aggregation_response.text
    

result = await parallel_tasks()
print(f"\n--- Агрегированное резюме ---\n{result}")

Затраченное время: 5.627488136291504 секунд

--- Индивидуальные результаты ---
Результат 1: ## Робот в джунглях

**Логлайн:** Веселый, но немного неуклюжий робот-ремонтник по имени Болт случайно вылетает с исследовательского судна и терпит крушение в ярких, неизведанных джунглях, где ему предстоит преодолеть свои конструктивные ограничения и научиться полагаться на причудливых существ, с которыми он подружился, чтобы выжить и найти дорогу домой.

**История:**

Болт, робот-санитар с блестящими глазами, писклявым голосом и склонностью к коллекционированию блестящих предметов, жил предсказуемой жизнью, убирая палубы исследовательского судна «Горизонт». Во время особенно сильного шторма волна-убийца сбрасывает Болта за борт, и он падает в густые, неизведанные джунгли Амазонки. 

Его программирование здесь бесполезно. Паутина забивает его пылесос, лианы опутывают его колеса, а влажность наносит ущерб его схемам. Поначалу Болт в ужасе. Но его оптимистичное программирование заставляет его нах

## Паттерн рефлексии

<div style="display: flex; justify-content: center;">
 <img src="../assets/agentic-patterns/reflection.png" alt="Reflection" style="max-width: 600px;">
</div>


Агент оценивает свой собственный вывод и использует эту обратную связь для итеративного уточнения своего ответа. Этот шаблон также известен как Оценщик-Оптимизатор и использует цикл самокоррекции. Начальный LLM генерирует ответ или выполняет задачу. Второй шаг LLM (или даже тот же LLM с другой подсказкой) затем действует как рефлектор или оценщик, критикуя первоначальный вывод на соответствие требованиям или желаемому качеству. Эта критика (обратная связь) затем передается обратно, побуждая LLM произвести уточненный вывод. Этот цикл может повторяться до тех пор, пока оценщик не подтвердит, что требования выполнены или достигнут удовлетворительный результат.


Сферы применения:
*   Генерация кода: написание кода, его выполнение, использование сообщений об ошибках или результатов тестов в качестве обратной связи для исправления ошибок.
*   Написание и уточнение: создание черновика, размышление о его ясности и тоне, а затем его пересмотр.
*   Решение сложных проблем: создание плана, оценка его осуществимости и его уточнение на основе оценки.
*   Извлечение информации: поиск информации и использование оценщика LLM для проверки того, были ли найдены все необходимые детали, прежде чем представлять ответ.

In [38]:
import os
import json
from google import genai
from pydantic import BaseModel
import enum

# Настройте клиент (убедитесь, что GEMINI_API_KEY установлен в вашей среде)
client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

class EvaluationStatus(enum.Enum):
    PASS = "PASS"
    FAIL = "FAIL"

class Evaluation(BaseModel):
    evaluation: EvaluationStatus
    feedback: str
    reasoning: str

# --- Функция начальной генерации ---
def generate_poem(topic: str, feedback: str = None) -> str:
    prompt = f"Напишите короткое, четырехстрочное стихотворение о {topic}."
    if feedback:
        prompt += f"\nВключите эту обратную связь: {feedback}"
    
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        contents=prompt
    )
    poem = response.text.strip()
    print(f"Сгенерированное стихотворение:\n{poem}")
    return poem

# --- Функция оценки ---
def evaluate(poem: str) -> Evaluation:
    print("\n--- Оценка стихотворения ---")
    prompt_critique = f"""Критикуйте следующее стихотворение. Хорошо ли оно рифмуется? В нем ровно четыре строки? 
Креативно ли оно? Ответьте PASS или FAIL и предоставьте обратную связь.

Стихотворение:
{poem}
"""
    response_critique = client.models.generate_content(
        model='gemini-2.0-flash',
        contents=prompt_critique,
        config={
            'response_mime_type': 'application/json',
            'response_schema': Evaluation,
        },
    )
    critique = response_critique.parsed
    print(f"Статус оценки: {critique.evaluation}")
    print(f"Обратная связь по оценке: {critique.feedback}")
    return critique

# Цикл рефлексии   
max_iterations = 3
current_iteration = 0
topic = "робот учится рисовать"

# смоделированное стихотворение, которое не пройдет оценку
current_poem = "С гудящими схемами, холодный и яркий,\nМеталлическая рука теперь держит кисть"

while current_iteration < max_iterations:
    current_iteration += 1
    print(f"\n--- Итерация {current_iteration} ---")
    evaluation_result = evaluate(current_poem)

    if evaluation_result.evaluation == EvaluationStatus.PASS:
        print("\nОкончательное стихотворение:")
        print(current_poem)
        break
    else:
        current_poem = generate_poem(topic, feedback=evaluation_result.feedback)
        if current_iteration == max_iterations:
            print("\nДостигнуто максимальное количество итераций. Последняя попытка:")
            print(current_poem)



--- Итерация 1 ---

--- Оценка стихотворения ---
Статус оценки: EvaluationStatus.FAIL
Обратная связь по оценке: Стихотворение неполное, предоставлено только две строки. Поэтому его нельзя оценить на предмет рифмовки или соответствия четырехстрочной структуре. Хотя образ робота, держащего кисть, несколько креативен, произведение необходимо закончить, чтобы полностью оценить его креативность.
Сгенерированное стихотворение:
С гудящими схемами, металлическая рука,
Теперь сжимает кисть, яркая команда.
По холсту расцветают и кружатся цвета,
Новое творение в цифровом мире.

--- Итерация 2 ---

--- Оценка стихотворения ---
Статус оценки: EvaluationStatus.PASS
Обратная связь по оценке: Стихотворение хорошо рифмуется (схема рифмовки AABB). В нем ровно четыре строки. Образ робота, создающего искусство, креативен и увлекателен.

Окончательное стихотворение:
С гудящими схемами, металлическая рука,
Теперь сжимает кисть, яркая команда.
По холсту расцветают и кружатся цвета,
Новое творение в цифровом

## Паттерн использования инструментов

<div style="display: flex; justify-content: center;">
 <img src="../assets/agentic-patterns/tool-use.png" alt="Tool Use" style="max-width: 250px;">
</div>

LLM имеет возможность вызывать внешние функции или API для взаимодействия с внешним миром, получения информации или выполнения действий. Этот шаблон часто называют вызовом функций и является наиболее широко известным шаблоном. LLM предоставляются определения (имя, описание, схема ввода) доступных инструментов (функции, API, базы данных и т. д.). На основе запроса пользователя LLM может решить вызвать один или несколько инструментов, сгенерировав структурированный вывод (например, JSON), соответствующий требуемой схеме. Этот вывод используется для выполнения фактического внешнего инструмента/функции, и результат возвращается в LLM. Затем LLM использует этот результат для формулирования своего окончательного ответа пользователю. Это значительно расширяет возможности LLM за пределы его обучающих данных.

Сферы применения:
*   Бронирование встреч с помощью API календаря.
*   Получение котировок акций в реальном времени через финансовый API.
*   Поиск в векторной базе данных релевантных документов (RAG).
*   Управление устройствами умного дома.
*   Выполнение фрагментов кода.

In [47]:
import os
from google import genai
from google.genai import types

# Настройте клиент (убедитесь, что GEMINI_API_KEY установлен в вашей среде)
client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

# Определите объявление функции для модели
weather_function = {
    "name": "get_current_temperature",
    "description": "Получает текущую температуру для заданного местоположения.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "Название города, например, Сан-Франциско",
            },
        },
        "required": ["location"],
    },
}

# Функция-заполнитель для имитации вызова API
def get_current_temperature(location: str) -> dict:
    return {"temperature": "15", "unit": "Celsius"}

# Создайте объект конфигурации, как показано в примере пользователя
# Используйте client.models.generate_content с моделью, содержимым и конфигурацией
tools = types.Tool(function_declarations=[weather_function])
contents = ["Какая сейчас температура в Лондоне?"]
response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=contents,
    config = types.GenerateContentConfig(tools=[tools])
)

# Обработка ответа (проверка на вызов функции)
response_part = response.candidates[0].content.parts[0]
if response_part.function_call:
    function_call = response_part.function_call
    print(f"Вызываемая функция: {function_call.name}")
    print(f"Аргументы: {dict(function_call.args)}")

    # Выполнить функцию
    if function_call.name == "get_current_temperature":        
        # Вызвать фактическую функцию
        api_result = get_current_temperature(*function_call.args)
        # Добавить вызов функции и результат выполнения функции в содержимое
        follow_up_contents = [
            types.Part(function_call=function_call),
            types.Part.from_function_response(
                name="get_current_temperature",
                response=api_result
            )
        ]
        # Сгенерировать окончательный ответ
        response_final = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=contents + follow_up_contents,
            config=types.GenerateContentConfig(tools=[tools])
        )
        print(response_final.text)
    else:
        print(f"Ошибка: запрошен неизвестный вызов функции: {function_call.name}")
else:
    print("В ответе не найден вызов функции.")
    print(response.text)

Вызываемая функция: get_current_temperature
Аргументы: {'location': 'London'}
Сейчас в Лондоне 15 градусов по Цельсию.



## Паттерн планирования (оркестратор-работники)

<div style="display: flex; justify-content: center;">
 <img src="../assets/agentic-patterns/planning.png" alt="Planning" style="max-width: 250px;">
</div>


Центральный планировщик LLM разбивает сложную задачу на динамический список подзадач, которые затем делегируются специализированным рабочим агентам (часто с использованием использования инструментов) для выполнения. Этот шаблон пытается решить сложные проблемы, требующие многоэтапного рассуждения, путем создания первоначального плана. Этот план динамически генерируется на основе ввода пользователя. Затем подзадачи назначаются «рабочим» агентам, которые их выполняют, потенциально параллельно, если позволяют зависимости. «Оркестратор» или «синтезатор» LLM собирает результаты от рабочих, размышляет о том, была ли достигнута общая цель, и либо синтезирует окончательный вывод, либо потенциально инициирует шаг перепланирования, если это необходимо. Это снижает когнитивную нагрузку на любой отдельный вызов LLM, улучшает качество рассуждений, минимизирует ошибки и позволяет динамически адаптировать рабочий процесс. Ключевое отличие от маршрутизации заключается в том, что планировщик генерирует *многоэтапный план*, а не выбирает один следующий шаг.

Сферы применения:
*   Сложные задачи разработки программного обеспечения: разбиение «создания функции» на подзадачи планирования, кодирования, тестирования и документирования.
*   Исследования и создание отчетов: планирование таких шагов, как поиск литературы, извлечение данных, анализ и написание отчета.
*   Мультимодальные задачи: планирование шагов, включающих генерацию изображений, анализ текста и интеграцию данных.
*   Выполнение сложных запросов пользователей, таких как «Спланируйте 3-дневную поездку в Париж, забронируйте авиабилеты и отель в рамках моего бюджета».



In [48]:

import os
from google import genai
from pydantic import BaseModel, Field
from typing import List

# Настройте клиент (убедитесь, что GEMINI_API_KEY установлен в вашей среде)
client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

# Определите схему плана
class Task(BaseModel):
    task_id: int
    description: str
    assigned_to: str = Field(description="Какой тип работника должен справиться с этим? Например, Исследователь, Писатель, Кодер")

class Plan(BaseModel):
    goal: str
    steps: List[Task]

# Шаг 1: Сгенерировать план (планировщик LLM)
user_goal = "Написать короткий пост в блоге о преимуществах агентов ИИ."

prompt_planner = f"""
Создайте пошаговый план для достижения следующей цели. 
Назначьте каждый шаг гипотетическому типу работника (Исследователь, Писатель).

Цель: {user_goal}
"""

print(f"Цель: {user_goal}")
print("Генерация плана...")

# Используйте модель, способную к планированию и структурированному выводу
response_plan = client.models.generate_content(
    model='gemini-2.5-pro-preview-03-25',
    contents=prompt_planner,
    config={
        'response_mime_type': 'application/json',
        'response_schema': Plan,
    },
)

# Шаг 2: Выполнить план (оркестратор/работники - опущено для краткости) 
for step in response_plan.parsed.steps:
    print(f"Шаг {step.task_id}: {step.description} (Исполнитель: {step.assigned_to})")

Цель: Написать короткий пост в блоге о преимуществах агентов ИИ.
Генерация плана...
Шаг 1: Исследовать и определить ключевые преимущества и распространенные применения агентов ИИ. (Исполнитель: Исследователь)
Шаг 2: Собрать подтверждающие доказательства, примеры или статистику для выявленных преимуществ. (Исполнитель: Исследователь)
Шаг 3: Создать план поста в блоге, включая введение, основные моменты (преимущества с примерами) и заключение. (Исполнитель: Писатель)
Шаг 4: Написать черновик поста в блоге, подробно описывая каждое преимущество, используя результаты исследования и придерживаясь плана. (Исполнитель: Писатель)
Шаг 5: Просмотреть и отредактировать черновик на предмет ясности, точности, грамматики, стиля и вовлеченности. (Исполнитель: Писатель)


## Паттерн с несколькими агентами

<div style="display: flex;;style="display: flex;>
    <div><img src="../assets/agentic-patterns/multi-agent.png" alt="Multi-Agent" style="max-width: 400px;"></div>
    <div><img src="../assets/agentic-patterns/multi-agent-2.png" alt="Multi-Agent" style="max-width: 400px;"></div>
</div>

Несколько отдельных агентов, каждому из которых назначена определенная роль, персона или опыт, сотрудничают для достижения общей цели. Этот шаблон использует автономных или полуавтономных агентов. Каждый агент может иметь уникальную роль (например, менеджер проекта, кодер, тестировщик, критик), специализированные знания или доступ к определенным инструментам. Они взаимодействуют и сотрудничают, часто координируемые центральным агентом-«координатором» или «менеджером» (например, PM на диаграмме) или с использованием логики передачи, когда один агент передает управление другому агенту.


Сферы применения:
*   Моделирование дебатов или мозговых штурмов с различными персонажами ИИ.
*   Создание сложного программного обеспечения с участием агентов для планирования, кодирования, тестирования и развертывания.
*   Проведение виртуальных экспериментов или симуляций с агентами, представляющими разных участников.
*   Совместное написание или процессы создания контента.
  

Примечание: приведенный ниже пример является упрощенным примером того, как использовать шаблон с несколькими агентами с логикой передачи и структурированным выводом. Я рекомендую взглянуть на [LangGraph Multi-Agent Swarm](https://github.com/langchain-ai/langgraph-swarm-py) или [Crew AI](https://www.crewai.com/open-source)

In [56]:
from google import genai
from pydantic import BaseModel, Field

# Настройте клиент (убедитесь, что GEMINI_API_KEY установлен в вашей среде)
client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

# Определите схемы структурированного вывода
class Response(BaseModel):
    handoff: str = Field(default="", description="Имя/роль агента, которому нужно передать управление. Доступные агенты: «Агент ресторана», «Агент отеля»")
    message: str = Field(description="Сообщение ответа пользователю или контекст для следующего агента")

# Функция агента
def run_agent(agent_name: str, system_prompt: str, prompt: str) -> Response:
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        contents=prompt,
        config = {'system_instruction': f'Вы — {agent_name}. {system_prompt}', 'response_mime_type': 'application/json', 'response_schema': Response}
    )
    return response.parsed


# Определите системные подсказки для агентов
hotel_system_prompt = "Вы — агент по бронированию отелей. Вы занимаетесь ТОЛЬКО бронированием отелей. Если пользователь спрашивает о ресторанах, рейсах или о чем-либо еще, ответьте коротким сообщением о передаче, содержащим исходный запрос, и установите для поля «handoff» значение «Агент ресторана». В противном случае обработайте запрос отеля и оставьте поле «handoff» пустым."
restaurant_system_prompt = "Вы — агент по бронированию ресторанов. Вы обрабатываете рекомендации и бронирования ресторанов на основе запроса пользователя, указанного в подсказке."

# Подсказка должна быть о ресторане
initial_prompt = "Можете ли вы забронировать мне столик в итальянском ресторане на 2 человека сегодня вечером?"]
print(f"Первоначальный запрос пользователя: {initial_prompt}")

# Запустите первого агента (агента отеля), чтобы принудительно использовать логику передачи
output = run_agent("Агент отеля", hotel_system_prompt, initial_prompt)

# смоделируйте взаимодействие с пользователем, чтобы изменить подсказку и передачу
if output.handoff == "Агент ресторана":
    print("Сработала передача: Отель в ресторан")
    output = run_agent("Агент ресторана", restaurant_system_prompt, initial_prompt)
elif output.handoff == "Агент отеля":
    print("Сработала передача: Ресторан в отель")
    output = run_agent("Агент отеля", hotel_system_prompt, initial_prompt)

print(output.message)    

Первоначальный запрос пользователя: Можете ли вы забронировать мне столик в итальянском ресторане на 2 человека сегодня вечером?
Сработала передача: Отель в ресторан
Мне нужно знать город, в котором вы хотели бы поужинать, и время, на которое вы хотели бы забронировать столик. Можете ли вы предоставить эту информацию?


## Комбинирование и настройка этих паттернов

Важно помнить, что эти шаблоны — не жесткие правила, а гибкие строительные блоки. Реальные агентные системы часто сочетают в себе элементы из нескольких шаблонов. Агент планирования может использовать использование инструментов, а его работники могут использовать рефлексию. Система с несколькими агентами может использовать внутреннюю маршрутизацию для назначения задач.

Ключом к успеху любого приложения LLM, особенно сложных агентных систем, является эмпирическая оценка. Определите метрики, измерьте производительность, выявите узкие места или точки отказа и итерируйте свой дизайн. Сопротивляйтесь чрезмерному усложнению.

## Благодарности

Этот обзор был создан с помощью глубокого и ручного исследования, черпая вдохновение и информацию из нескольких отличных ресурсов, в том числе:
*   [5 шаблонов проектирования агентов ИИ](https://blog.dailydoseofds.com/p/5-agentic-ai-design-patterns)
*   [Что такое агентные рабочие процессы?](https://weaviate.io/blog/what-are-agentic-workflows)
*   [Создание эффективных агентов](https://www.anthropic.com/engineering/building-effective-agents)
*   [Как агенты могут улучшить производительность LLM](https://www.deeplearning.ai/the-batch/how-agents-can-improve-llm-performance)
*   [Шаблоны проектирования агентов](https://medium.com/@bijit211987/agentic-design-patterns-cbd0aae2962f)
*   [Рецепты агентов](https://www.agentrecipes.com/)
*   [Концепции агентов LangGraph](https://langchain-ai.github.io/langgraph/concepts/agentic_concepts/)
*   [Примеры агентов OpenAI на Python](https://github.com/openai/openai-agents-python/tree/main/examples/agent_patterns)
*   [Поваренная книга Anthropic](https://github.com/anthropics/anthropic-cookbook/blob/main/patterns/agents)